Get the MI, the MI mean and the Pathologic pairs

estimated time: 52 min

In [1]:
#IPython extension to reload modules before executing user code.
#'autoreload' reloads modules automatically before entering the execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import gpustat

#select the best free GPU on the nvidia card
stats = gpustat.GPUStatCollection.new_query()
ids = map(lambda gpu: int(gpu.entry['index']), stats)
ratios = map(lambda gpu: float(gpu.entry['memory.used'])/float(gpu.entry['memory.total']), stats)
bestGPU = min(zip(ids, ratios), key=lambda x: x[1])[0]
bestGPU = 0

print("setGPU: Setting GPU to: {}".format(bestGPU))
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(bestGPU)


#set memory usage to 0.5
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.66
set_session(tf.Session(config=config))

setGPU: Setting GPU to: 0


Using TensorFlow backend.


In [3]:
#!pip install -q sklearn
import collections
import numpy as np, numpy
from keract import get_activations, display_activations
import keras
from keras import backend as K
from keras import optimizers, regularizers
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10, cifar100 # we can use also cifar100
from keras.layers import Input, BatchNormalization, AveragePooling2D, ZeroPadding2D, LeakyReLU, GlobalAveragePooling2D, Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model, Model
from keras.regularizers import l2
from keras.utils import plot_model, to_categorical
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import scipy
import sklearn
import sklearn.metrics
import sys
#sys.executable
#sys.path
import time
from IPython.display import SVG

from layca_optimizers import SGD

In [4]:
def normalize(x, mean, std):
    # This function normalizes inputs for zero mean and unit variance to speed up learning.
    
    # In case std = 0, we add eps = 1e-7
    eps = K.epsilon()
    x = (x-mean)/(std+eps)
    return x
  
def import_cifar(dataset):
    if dataset == 10:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    elif dataset == 100:
        (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

    # By default, they are uint8 but we need them float to normalize them
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    # Calculating the mean and standard deviation of the training data
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    
    # Normalizing 
    x_train = normalize(x_train, mean, std)
    x_test = normalize(x_test, mean, std)

    y_train = to_categorical(y_train, num_classes=dataset)
    y_test = to_categorical(y_test,  num_classes=dataset)
    
    return (x_train, y_train), (x_test, y_test)

In [ ]:
# LOAD DATABase
num_classes = 10
(x_train, y_train), (x_test, y_test) = import_cifar(num_classes)

In [ ]:
# Architecture taken from https://github.com/geifmany/cifar-vgg
# Weight decay and Dropout have been removed
# BatchNormalization before activations
def VGG16_Vanilla_beta(input_shape, num_classes):
    model = Sequential([
        #0
        Conv2D(64, (3, 3), padding='same', input_shape=input_shape),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #3
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #7
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #10
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #14
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #17
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #20
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #24
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #27
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #30
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        #34
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #37
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #40
        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        #45
        Dense(512),
        BatchNormalization(scale=False, center=False),
        Activation('relu'),
        #48
        Dense(num_classes),
        BatchNormalization(scale=False, center=False),
        Activation('softmax')])
    return model

In [ ]:
# EVALUATION
# Final evaluation of the models
#scores = model.evaluate(x_test, y_test, verbose=1)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])
#print("%.2f%% : Model Error" % (100-scores[1]*100))

In [ ]:
List_of_Pré_Activations = ['conv2d_1/BiasAdd:0','conv2d_2/BiasAdd:0','max_pooling2d_1/MaxPool:0','conv2d_3/BiasAdd:0', 'conv2d_4/BiasAdd:0','max_pooling2d_2/MaxPool:0','conv2d_5/BiasAdd:0', 'conv2d_6/BiasAdd:0','conv2d_7/BiasAdd:0', 'max_pooling2d_3/MaxPool:0','conv2d_8/BiasAdd:0', 'conv2d_9/BiasAdd:0','conv2d_10/BiasAdd:0', 'max_pooling2d_4/MaxPool:0','conv2d_11/BiasAdd:0', 'conv2d_12/BiasAdd:0','conv2d_13/BiasAdd:0', 'max_pooling2d_5/MaxPool:0','dense_1/BiasAdd:0', 'dense_2/BiasAdd:0']
List_of_Post_Activations = [    'activation_1/Relu:0','activation_2/Relu:0',    'activation_3/Relu:0','activation_4/Relu:0',    'activation_5/Relu:0','activation_6/Relu:0',    'activation_7/Relu:0','activation_8/Relu:0',    'activation_9/Relu:0','activation_10/Relu:0',    'activation_11/Relu:0','activation_12/Relu:0',    'activation_13/Relu:0','activation_14/Relu:0',    'activation_15/Softmax:0']

In [ ]:
def MI(IM, nameOfTheLayer):
    
    #count of the number of MI = 1
    CountMI1 = 0
    #count the number of pathologic neurons (only 1 or 0)
    pathologicCount = 0
    pathologicCount_00 = 0
    pathologicCount_01 = 0
    pathologicCount_11 = 0
    MutualInfoWithoutOnes = []
    
    #Activati0ns
    Activati0nsList = []
    
    #nombre d'images utilisées pour l'activation: nombreDImagesDActivation
    
    #nombre de neurones dans la couche
    numNeurons = numpy.size(IM[nameOfTheLayer][0])
    print('number of neurons for the layer: '+str(numNeurons))
    
    # on selectionne des pair de neurones (X,Y) à comparer (regarder l'information mutuelle entre X et Y)
    NeuronsSelected = numpy.zeros((NombrePairs,2))  # initialize
    for i in range(NombrePairs):
        NeuronsSelected[i] = numpy.ceil(numpy.random.rand(2)*numNeurons)-1  # on prend 2 neurones selectionnées aleatoirement
        # les 2 neurones doivent être différents
        while NeuronsSelected[i][0] == NeuronsSelected[i][1]:
            NeuronsSelected[i] = numpy.ceil(numpy.random.rand(2)*numNeurons)-1
    #print(NeuronsSelected[56])
    
    MutualInfo = numpy.zeros((NombrePairs,1))  # initialization des resultats
    
    #creation de la matrice des activations
    MatrixActivation = np.zeros((numNeurons,nombreDImagesDActivation))
    
    indexActi = -1
    for activation in IM[nameOfTheLayer]:   # on a les activations pour de la couche
        indexActi = indexActi + 1
        activation = activation.flatten() # on a tous nos neurones sur un seul vecteur
        for i in range(numNeurons):
            MatrixActivation[i,indexActi] = activation[i]
                                
    #print(numpy.size(MatrixActivation[0,:])) #100
    #print(numpy.size(MatrixActivation[:,0])) #65534

    #on binarise les activations par rapport à la moyenne de l'activation du neurone
    for j in range(numNeurons):
        neuronMean = numpy.mean(MatrixActivation[j,:])
        for i in range(nombreDImagesDActivation):
            if MatrixActivation[j,i] > neuronMean:
                MatrixActivation[j,i] = 1
            else:
                MatrixActivation[j,i] = 0
                
    #on s'en fou de les normer vu qu'on binarise à la moyenne
                
    X= numpy.zeros((1,nombreDImagesDActivation))
    Y= numpy.zeros((1,nombreDImagesDActivation))
    
    #on va calculer la MI de 2 neurones differents:
    for j in range(NombrePairs):
        X= MatrixActivation[int(NeuronsSelected[j,0]),:]
        Y= MatrixActivation[int(NeuronsSelected[j,1]),:]
        # example:= sklearn.metrics.normalized_mutual_info_score([1.1,1.0,1.0,1.0,0.0],[1.1,0.1,0.1,1.1,0.1])
        MutualInfo[j] = sklearn.metrics.normalized_mutual_info_score(X,Y,average_method='max')
        #print(MutualInfo[j])
        
        # on capte les neurones pathologiques
        if (MutualInfo[j])==1. or (MutualInfo[j])==0.:
            CountMI1 = CountMI1 + 1
            if all([ v == 0 for v in X ]) and all([ v == 0 for v in Y ]):
                pathologicCount = pathologicCount + 1
                pathologicCount_00 = pathologicCount_00 + 1
            elif all([ v == 1 for v in X ]) and all([ v == 1 for v in Y ]):
                pathologicCount = pathologicCount + 1
                pathologicCount_11 = pathologicCount_11 + 1
            elif all([ v == 0 for v in X ]) and all([ v == 1 for v in Y ]):
                pathologicCount = pathologicCount + 1
                pathologicCount_01 = pathologicCount_01 + 1
            elif all([ v == 1 for v in X ]) and all([ v == 0 for v in Y ]):
                pathologicCount = pathologicCount + 1
                pathologicCount_01 = pathologicCount_01 + 1
        else:
            MutualInfoWithoutOnes.append(MutualInfo[j])
            
        # nombre de 0 dans les activations
        Activati0nsList.append(len(X)-np.count_nonzero(X))
        Activati0nsList.append(len(Y)-np.count_nonzero(Y))
    Activati0ns = np.mean(Activati0nsList)/len(X)
    
    return MutualInfo, MutualInfoWithoutOnes, CountMI1, pathologicCount, pathologicCount_00, pathologicCount_11, pathologicCount_01, Activati0ns

In [ ]:
def RUN(IM, modelename):
    VarPath = []
    MIMean = []
    MI_1Mean = []
    PCount = []
    PCount_0 = []
    PCount_1 = []
    PCount_M = []
    #Pourcentage de 0 dans les activations
    Activati0ns = []
    
    ###############################""
    
    print('number of images used for see the activations : '+str(nombreDImagesDActivation) + '\n')
    print('number of pairs of neurons analyzed: '+str(NombrePairs))

    fig = plt.figure(figsize=(10, 60))
    plt.ylim(top=1000)
    plt.rcParams.update({'font.size': 16})

    print('\n Model :\n')
    i = 1
    for layer in List_of_Post_Activations:
        plt.subplot(20,1,i)
        i = i+1
        #nom de la couche
        print('couche :', str(layer))
        #MI
        MutualInfo, MutualInfoWithoutOnes, CountMI1, pathologicCount, pathologicCount_00, pathologicCount_11, pathologicCount_01, Activati0n = MI(IM, layer)
        MIMean.append(numpy.mean(MutualInfo))
        MI_1Mean.append(numpy.mean(MutualInfoWithoutOnes))
        PCount.append(pathologicCount)
        PCount_0.append(pathologicCount_00)
        PCount_1.append(pathologicCount_11)
        PCount_M.append(pathologicCount_01)
        Activati0ns.append(Activati0n)
        #plot
        plt.hist(MutualInfo, range = (0, 1.001), bins = 200, color = 'yellow',
            edgecolor = 'red')
        plt.xlabel('Mutual Information')
        plt.ylabel('Occurence')
        plt.grid()
        #plt.title('\n Occurence of the mutual information post-activation of '+str(nombreDImagesDActivation)+' activation images\n over '+str(NombrePairs)+' pairs of neurons taken randomly\n on the layer '+ str(layer)+' of the model '+str(modelename)+'\n')
        plt.title(str(modelename)+' : '+str(layer)+'\n')
        plt.tight_layout()
        plt.ylim(top=NombrePairs)
        plt.axvline(x=MIMean[-1], color='red')
        plt.axvline(x=MI_1Mean[-1], color='blue')
        #plt.legend('MI mean', 'MI mean without 1')
        print('MI mean: ', str(MIMean[-1]))
        print('MI without_1 mean: ', str(MI_1Mean[-1]))
        print('pathologic pair Count: ', str(pathologicCount),' on ',str(NombrePairs),' pairs = ',str(pathologicCount/NombrePairs*100),'%')
        print('pathologic pair Count: ', str(pathologicCount),'->  pathologic Count 0: ',str(pathologicCount_00),'  pathologic Count 1: ',str(pathologicCount_11),'  pathologic Count mix: ', str(pathologicCount_01))
        if CountMI1 != 0:
            VarPath.append(pathologicCount/CountMI1)
        else:
            VarPath.append(0)
        print('pathologic pair Count: ', str(pathologicCount),' on ',str(CountMI1),' MI1 = ',str(VarPath[-1]*100),'%')
        print('\n')
    plt.show()

    fig.savefig('figures/Activations/MI_{}.png'.format(modelename), dpi=100)
    #################################""
    
    # plot:
    plt.rcParams.update({'font.size': 20})
    fig = plt.figure(figsize=(10, 5))
    plt.ylim(top=0.2)
    nombreCouches = len(MIMean)
    x = range(nombreCouches)

    markerline, stemlines, baseline = plt.stem(x, MIMean, markerfmt='o', label='model ')
    plt.setp(stemlines, 'color', plt.getp(markerline,'color'))
    plt.setp(stemlines, 'linestyle', 'dotted')
    line, = plt.plot(np.mean(MIMean)*np.ones((nombreCouches,1)))

    '''
    for i in range(len(Activati0ns)):
        if isinstance(Activati0ns[i], float):
            Activati0ns[i] = str("%.2f" % Activati0ns[i])
        text = plt.text(-0.33+i, 0.01, Activati0ns[i], rotation=90)'''

    #extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    #plt.legend([markerline, line], ['MI by layer', 'MI mean'])
    #plt.xlabel('Layers')
    #plt.xticks(np.arange(nombreCouches), ('Conv','Conv','MaxPool','Conv','Conv','MaxPool','Conv','Conv','Conv','MaxPool','Conv','Conv','Conv','MaxPool','Conv','Conv','Conv','MaxPool','Dense','Dense'), rotation=60)
    plt.xticks(np.arange(nombreCouches), ('Conv 1','Conv 2','Conv 3','Conv 4','Conv 5','Conv 6','Conv 7','Conv 8','Conv 9','Conv10','Conv11','Conv12','Conv13','Dense1','Dense2'), rotation=60)
    #plt.ylabel('MI mean')
    plt.grid()
    #plt.title('MI mean post-activation by layer of the model '+modelename+' \n for '+str(nombreDImagesDActivation)+' activation images and '+str(NombrePairs)+ ' pairs of neurons\n')
    plt.show()
    fig.savefig('figures/Activations/MIMean_{}.png'.format(modelename), dpi=100)
    
    #################################
    
    #Ratio of 0 in the activations
    '''fig = plt.figure(figsize=(10, 5))
    plt.ylim(top=1)
    markerline, stemlines, baseline = plt.stem(x, Activati0ns, markerfmt='o', label='model ')
    plt.legend([markerline], ['Ratio by layer'])
    plt.xlabel('Layers')
    #plt.xticks(np.arange(nombreCouches), ('Conv','Conv','MaxPool','Conv','Conv','MaxPool','Conv','Conv','Conv','MaxPool','Conv','Conv','Conv','MaxPool','Conv','Conv','Conv','MaxPool','Dense','Dense'), rotation=60)
    plt.xticks(np.arange(nombreCouches), ('Conv 1','Conv 2','Conv 3','Conv 4','Conv 5','Conv 6','Conv 7','Conv 8','Conv 9','Conv10','Conv11','Conv12','Conv13','Dense1','Dense2'), rotation=60)
    plt.ylabel('Ratio')
    plt.title('Ratio of the mean of 0 by layer of the model '+modelename+'\n for '+str(nombreDImagesDActivation)+' activation images on '+str(2*NombrePairs)+ ' neurons')
    plt.show()
    fig.savefig('figures/0Ratio_{}.png'.format(modelename), dpi=100)'''

    file = open('figures/Activations/0Ratio_{}.txt'.format(modelename),'w') 
    for i in range(len(Activati0ns)):
        if isinstance(Activati0ns[i], float):
            Activati0ns[i] = str("%.2f" % Activati0ns[i])
        file.write(Activati0ns[i])
        file.write('/')
    file.close() 
    
    ###############################
    
    # plot: proportion de pairs de neurones pathologiques
    fig = plt.figure(figsize=(10, 6))
    plt.ylim(top=1400)

    plt.bar(range(len(PCount_0)),PCount_0, width=-0.2, align='edge', color='red')
    plt.bar(range(len(PCount)),PCount, width=0.2, align='center', color='yellow')
    plt.bar(range(len(PCount_1)),PCount_1, width=0.2, align='edge', color='blue')
    plt.bar(range(len(PCount_M)),PCount_M, width=0.2, align='center', color='green')
    #plt.legend(['Pathologic_00', 'Pathologic','Pathologic_11','Pathologic_Mix'])
    #plt.xlabel('Layers')
    plt.xticks(np.arange(nombreCouches), ('Conv 1','Conv 2','Conv 3','Conv 4','Conv 5','Conv 6','Conv 7','Conv 8','Conv 9','Conv10','Conv11','Conv12','Conv13','Dense1','Dense2'), rotation=60)
    #plt.ylabel('Nomber')
    #plt.title('Nomber of pairs of pathological neurons post-activation for the model '+modelename+' \n for '+str(nombreDImagesDActivation)+' activation images and '+str(NombrePairs)+ ' pairs of neurons\n')
    plt.grid()
    plt.show()

    fig.savefig('figures/Activations/MIPatho_{}.png'.format(modelename), dpi=100)
    # Résultat logique quand on regarde le nombre de neurones pathologiques (environ 80/10000 pairs patho car 9/100 neurones patho)

In [ ]:
# LOAD the model and get the activations
modelename = 'vgg16_wdecay_0'
model_name = '../weights/6/{}/initial/weights-initial.hdf5'.format(modelename) #final / initial ! attention au nom de sauvegarde !
modelename = 'initial'
model = VGG16_Vanilla_beta(input_shape=(32,32,3), num_classes=10)
model.load_weights('{}'.format(model_name))
sgd = optimizers.SGD(lr=0.002, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# On regarde l'activation des pairs de neurones pour un echantillon test pris au hasard
Echantillon = []
nombreDImagesDActivation = 1000
NombrePairs = 10000
choix = np.random.choice(x_test.shape[0], nombreDImagesDActivation)
Echantillon = x_train[choix, :, : , :]
IM = get_activations(model, Echantillon)# on affiche les noms des differentes couches
# Run the code
RUN(IM, modelename)

number of images used for see the activations : 1000

number of pairs of neurons analyzed: 10000

 Model :

couche : activation_1/Relu:0
number of neurons for the layer: 65536
0.4
0.305
0.347
0.343
0.263
0.342
0.306
0.261
0.378
0.259
0.323
0.289
0.342
0.387
0.341
0.306
0.365
0.375
0.331
0.315
0.289
0.382
0.31
0.441
0.397
0.41
0.367
0.264
0.355
0.379
0.347
0.368
0.36
0.277
0.408
0.365
0.312
0.358
0.316
0.325
0.38
0.401
0.346
0.337
0.369
0.317
0.366
0.37
0.424
0.33
0.282
0.428
0.395
0.342
0.311
0.268
0.376
0.344
0.317
0.272
0.4
0.3
0.302
0.307
0.403
0.259
0.349
0.367
0.415
0.352
0.364
0.364
0.345
0.26
0.324
0.338
0.357
0.34
0.342
0.303
0.368
0.343
0.351
0.372
0.35
0.402
0.349
0.366
0.344
0.378
0.392
0.368
0.36
0.316
0.367
0.384
0.372
0.367
0.404
0.353
0.382
0.311
0.293
0.365
0.393
0.38
0.369
0.445
0.332
0.354
0.332
0.378
0.303
0.248
0.331
0.347
0.358
0.237
0.411
0.29
0.385
0.325
0.31
0.318
0.291
0.337
0.288
0.307
0.408
0.4
0.341
0.327
0.37
0.376
0.252
0.361
0.362
0.349
0.323
0.314
0.289


0.432
0.323
0.39
0.352
0.317
0.475
0.334
0.374
0.281
0.309
0.358
0.348
0.326
0.419
0.294
0.405
0.372
0.356
0.351
0.35
0.342
0.356
0.261
0.366
0.329
0.346
0.369
0.361
0.347
0.388
0.366
0.367
0.393
0.373
0.267
0.302
0.34
0.323
0.431
0.386
0.317
0.328
0.248
0.296
0.349
0.38
0.276
0.369
0.349
0.355
0.382
0.369
0.361
0.329
0.356
0.334
0.406
0.439
0.365
0.332
0.381
0.418
0.434
0.284
0.391
0.338
0.379
0.339
0.425
0.406
0.315
0.394
0.283
0.292
0.347
0.356
0.431
0.344
0.296
0.361
0.433
0.408
0.407
0.328
0.347
0.419
0.321
0.395
0.428
0.397
0.4
0.333
0.398
0.359
0.355
0.392
0.36
0.371
0.425
0.372
0.443
0.37
0.331
0.411
0.261
0.366
0.38
0.338
0.37
0.323
0.335
0.287
0.406
0.371
0.363
0.365
0.389
0.335
0.392
0.342
0.299
0.384
0.415
0.326
0.273
0.352
0.336
0.34
0.285
0.373
0.401
0.365
0.332
0.35
0.4
0.395
0.336
0.309
0.307
0.35
0.257
0.35
0.332
0.372
0.329
0.337
0.402
0.38
0.324
0.449
0.351
0.34
0.306
0.404
0.357
0.355
0.403
0.29
0.338
0.433
0.428
0.349
0.336
0.342
0.297
0.335
0.267
0.271
0.344
0.344

0.383
0.282
0.407
0.352
0.289
0.388
0.276
0.328
0.342
0.25
0.402
0.318
0.359
0.378
0.397
0.409
0.346
0.416
0.379
0.409
0.32
0.297
0.386
0.374
0.39
0.384
0.392
0.423
0.315
0.437
0.336
0.358
0.384
0.341
0.269
0.323
0.296
0.383
0.266
0.292
0.378
0.322
0.343
0.361
0.356
0.333
0.344
0.375
0.338
0.293
0.394
0.301
0.321
0.35
0.358
0.35
0.394
0.33
0.347
0.301
0.322
0.311
0.283
0.358
0.299
0.387
0.388
0.394
0.447
0.311
0.314
0.355
0.358
0.418
0.299
0.358
0.34
0.344
0.277
0.276
0.278
0.331
0.385
0.308
0.389
0.329
0.344
0.365
0.392
0.341
0.372
0.279
0.343
0.374
0.38
0.398
0.326
0.282
0.363
0.316
0.334
0.369
0.368
0.366
0.417
0.365
0.323
0.32
0.465
0.314
0.326
0.279
0.427
0.308
0.277
0.348
0.37
0.267
0.368
0.33
0.338
0.341
0.326
0.404
0.378
0.415
0.302
0.348
0.443
0.378
0.402
0.335
0.361
0.355
0.354
0.385
0.387
0.278
0.358
0.351
0.334
0.445
0.389
0.442
0.333
0.415
0.398
0.373
0.37
0.371
0.306
0.364
0.316
0.31
0.29
0.391
0.353
0.439
0.393
0.316
0.368
0.35
0.34
0.316
0.354
0.329
0.417
0.347
0.306
0.

0.29
0.391
0.467
0.387
0.428
0.38
0.402
0.397
0.295
0.393
0.357
0.374
0.368
0.368
0.368
0.313
0.35
0.339
0.451
0.298
0.379
0.329
0.29
0.292
0.351
0.292
0.326
0.375
0.284
0.328
0.445
0.367
0.347
0.355
0.336
0.371
0.355
0.324
0.306
0.348
0.404
0.302
0.295
0.323
0.263
0.446
0.365
0.275
0.402
0.415
0.354
0.361
0.293
0.383
0.323
0.334
0.307
0.344
0.335
0.342
0.342
0.362
0.384
0.369
0.341
0.342
0.427
0.358
0.329
0.381
0.385
0.34
0.348
0.274
0.345
0.376
0.257
0.33
0.373
0.383
0.303
0.363
0.417
0.297
0.319
0.335
0.303
0.447
0.304
0.394
0.288
0.397
0.359
0.36
0.313
0.288
0.345
0.341
0.27
0.261
0.356
0.409
0.365
0.285
0.328
0.276
0.342
0.267
0.359
0.396
0.394
0.37
0.342
0.398
0.349
0.366
0.4
0.364
0.362
0.355
0.309
0.395
0.324
0.412
0.39
0.255
0.393
0.359
0.308
0.336
0.334
0.427
0.279
0.295
0.286
0.3
0.305
0.297
0.384
0.377
0.368
0.36
0.368
0.3
0.348
0.359
0.387
0.356
0.372
0.38
0.391
0.329
0.356
0.39
0.334
0.452
0.457
0.4
0.406
0.37
0.377
0.364
0.279
0.371
0.381
0.411
0.3
0.353
0.308
0.33
0.329

0.332
0.352
0.378
0.378
0.364
0.396
0.359
0.361
0.344
0.394
0.31
0.284
0.274
0.325
0.294
0.338
0.304
0.38
0.377
0.347
0.35
0.337
0.348
0.301
0.314
0.338
0.328
0.363
0.383
0.307
0.295
0.26
0.318
0.362
0.425
0.309
0.33
0.426
0.364
0.395
0.344
0.288
0.324
0.368
0.348
0.37
0.43
0.318
0.248
0.392
0.34
0.383
0.389
0.374
0.312
0.384
0.368
0.336
0.374
0.331
0.333
0.397
0.339
0.379
0.44
0.446
0.338
0.319
0.304
0.301
0.374
0.294
0.463
0.394
0.324
0.385
0.329
0.387
0.249
0.359
0.298
0.345
0.309
0.366
0.379
0.333
0.297
0.352
0.386
0.355
0.355
0.346
0.371
0.268
0.336
0.346
0.36
0.347
0.413
0.277
0.33
0.35
0.356
0.347
0.382
0.285
0.335
0.402
0.358
0.381
0.425
0.34
0.37
0.368
0.351
0.354
0.365
0.348
0.336
0.443
0.404
0.398
0.394
0.309
0.303
0.362
0.354
0.338
0.333
0.289
0.377
0.402
0.366
0.306
0.351
0.4
0.412
0.348
0.327
0.409
0.377
0.438
0.343
0.399
0.277
0.307
0.348
0.388
0.281
0.319
0.35
0.345
0.339
0.358
0.331
0.349
0.277
0.265
0.351
0.289
0.343
0.33
0.367
0.438
0.356
0.398
0.345
0.359
0.443
0.31

0.399
0.33
0.255
0.329
0.384
0.368
0.349
0.35
0.368
0.301
0.321
0.296
0.43
0.335
0.376
0.321
0.329
0.402
0.302
0.344
0.336
0.41
0.4
0.347
0.263
0.372
0.32
0.371
0.432
0.429
0.367
0.401
0.372
0.351
0.415
0.352
0.363
0.329
0.31
0.301
0.39
0.327
0.354
0.34
0.368
0.334
0.403
0.37
0.381
0.293
0.368
0.37
0.3
0.395
0.362
0.352
0.357
0.42
0.402
0.347
0.381
0.222
0.352
0.422
0.36
0.365
0.336
0.331
0.335
0.336
0.325
0.279
0.321
0.397
0.303
0.416
0.441
0.367
0.279
0.307
0.249
0.351
0.327
0.308
0.35
0.307
0.382
0.314
0.403
0.312
0.38
0.363
0.357
0.39
0.349
0.315
0.292
0.324
0.415
0.398
0.357
0.459
0.321
0.36
0.33
0.367
0.35
0.326
0.353
0.344
0.282
0.356
0.362
0.363
0.35
0.327
0.364
0.333
0.41
0.401
0.426
0.39
0.328
0.328
0.434
0.338
0.379
0.308
0.357
0.333
0.379
0.312
0.404
0.388
0.387
0.349
0.372
0.383
0.329
0.278
0.314
0.391
0.291
0.337
0.32
0.404
0.369
0.288
0.337
0.323
0.384
0.359
0.343
0.363
0.3
0.355
0.401
0.404
0.339
0.323
0.296
0.388
0.335
0.301
0.357
0.348
0.352
0.382
0.354
0.293
0.378
0.

0.352
0.362
0.313
0.424
0.374
0.353
0.357
0.315
0.352
0.284
0.361
0.365
0.319
0.398
0.312
0.371
0.354
0.297
0.367
0.352
0.338
0.37
0.35
0.416
0.328
0.344
0.317
0.297
0.342
0.428
0.367
0.353
0.347
0.287
0.349
0.32
0.289
0.341
0.371
0.332
0.343
0.259
0.312
0.28
0.36
0.269
0.355
0.358
0.314
0.338
0.269
0.333
0.351
0.303
0.34
0.339
0.324
0.37
0.37
0.397
0.36
0.351
0.358
0.363
0.295
0.312
0.38
0.338
0.381
0.411
0.396
0.34
0.356
0.395
0.394
0.353
0.391
0.321
0.37
0.314
0.357
0.335
0.303
0.345
0.316
0.336
0.355
0.36
0.347
0.385
0.361
0.455
0.384
0.399
0.32
0.442
0.367
0.278
0.314
0.368
0.395
0.357
0.363
0.344
0.329
0.296
0.359
0.301
0.321
0.354
0.376
0.4
0.372
0.381
0.334
0.321
0.33
0.373
0.335
0.336
0.338
0.322
0.394
0.28
0.362
0.34
0.324
0.307
0.361
0.275
0.402
0.357
0.31
0.34
0.346
0.308
0.268
0.408
0.406
0.473
0.296
0.373
0.327
0.334
0.364
0.377
0.403
0.307
0.26
0.297
0.365
0.4
0.287
0.401
0.4
0.305
0.348
0.325
0.345
0.411
0.378
0.372
0.334
0.336
0.357
0.339
0.38
0.264
0.31
0.245
0.363
0.

In [ ]:
modelename = 'vgg16_layca_0'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_layca_1'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_layca_2'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_sgd_0'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_sgd_1'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_sgd_2'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_wdecay_0'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_wdecay_1'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)

In [ ]:
modelename = 'vgg16_wdecay_2'
model_name = '../weights/6/{}/final/weights-final.hdf5'.format(modelename)
model.load_weights('{}'.format(model_name))
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
IM = get_activations(model, Echantillon)
# Run the code
RUN(IM, modelename)